## Credit Card Fraud Detection - Task3

### Load transaction data

In [6]:
import pandas as pd
import numpy as np

# Read transaction data from log file
transaction_data = pd.read_csv("data/task3_data.csv")
transaction_data.head()

,DateTime,CardID,TerminalID,Amount,Class,year,month,day,hour,minute,CardKind,CardLimit,CardHomeLocation,TerminalKind,TerminalMerchant,TerminalLocation
0,2018-06-09 15:26:00,546708,41,365,0,2018,6,9,15,26,0,0,54,0,0,4
1,2018-06-09 15:26:00,189780,187,635,1,2018,6,9,15,26,0,0,18,0,0,18
2,2018-06-09 15:26:00,11605758,15044,3865,0,2018,6,9,15,26,1,1,60,1,5,4
3,2018-06-09 15:26:00,550284,15551,119,0,2018,6,9,15,26,0,0,55,1,5,55
4,2018-06-09 15:26:00,502578,344,110,0,2018,6,9,15,26,0,0,50,0,0,34


### Train/Test(Validation) Split

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = transaction_data.drop(['Class', 'DateTime', 'CardID', 'TerminalID'], axis=1)
y = transaction_data['Class']

scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(211, 12) (211,)
(53, 12) (53,)


### Build Prediction Model Using Chosen Classification Algorithm

In [8]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### Test Prediction Model

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

y_test_predicted = model.predict(X_test)

accuracy = accuracy_score(y_test, y_test_predicted)

print('*****Test*****')
print("Accuracy = %.2f" % accuracy)
print(classification_report(y_test, y_test_predicted))

*****Test*****
Accuracy = 0.79
             precision    recall  f1-score   support

          0       0.86      0.91      0.88        46
          1       0.00      0.00      0.00         7

avg / total       0.74      0.79      0.77        53



In [10]:
from sklearn.metrics import confusion_matrix

results = confusion_matrix(y_test, y_test_predicted)
tn, fp, fn, tp = results.ravel()
print('Confusion matrix:')
print(results)
print("True Positives (TP) = %d" % tp)
print("True Negatives (TN) = %d" % tn)
print("False Positives (FP) = %d (Type1 error)" % fp)
print("False Negatives (FN) = %d (Type2 error)" % fn)

Confusion matrix:
[[42  4]
 [ 7  0]]
True Positives (TP) = 0
True Negatives (TN) = 42
False Positives (FP) = 4 (Type1 error)
False Negatives (FN) = 7 (Type2 error)
